# 1. 读取数据

读取txt文件，转化为npy格式
这一部分代码使用本地电脑运行。

In [ ]:
import itertools
import os
import numpy as np
import dataset


In [4]:
## 不同轮次的输入数据可放在不同文件夹中便于管理，这里用户可以自定义
PathSet = {0: "./Test", 1: "./CompetitionData1", 2: "./CompetitionData2", 3: "./CompetitionData3"}
PrefixSet = {0: "Round0", 1: "Round1", 2: "Round2", 3: "Round3"}

Ridx = 1 # 设置比赛轮次索引，指明数据存放目录。0:Test; 1: 1st round; 2: 2nd round ...

PathRaw = PathSet[Ridx]
Prefix = PrefixSet[Ridx]

# 查找文件夹中包含的所有比赛/测试数据文件，非本轮次数据请不要放在目标文件夹中
files = os.listdir(PathRaw)
names = []
for f in sorted(files):
    if f.find('CfgData') != -1 and f.endswith('.txt'):
        names.append(f.split('CfgData')[-1].split('.txt')[0])

## 创建对象并处理
for Didx in names:
    ## 创建对象并处理
    print('Processing Round ' + str(Ridx) + ' Case ' + Didx)
    
    # 读取配置文件 RoundYCfgDataX.txt
    print('Loading configuration data file')
    cfg_path = PathRaw + '/' + Prefix + 'CfgData' + Didx + '.txt'
    bs_pos, tol_samp_num, anch_samp_num, port_num, ant_num, sc_num = dataset.read_cfg_file(cfg_path)
    
    # 读取锚点位置文件 RoundYInputPosX.txt
    print('Loading input position file')
    anch_pos_path = PathRaw + '/' + Prefix + 'InputPos' + Didx + '.txt'
    anch_pos = dataset.read_anch_file(anch_pos_path, anch_samp_num)
    
    csi_file_npy = PathRaw + '/' + Prefix + 'InputData' + Didx + '.npy'
    csi_file_txt = PathRaw + '/' + Prefix + 'InputData' + Didx + '.txt'
    
    if os.path.isfile(csi_file_npy):
        # 读取信道文件 RoundYInputDataX.txt，这里已经读取到了.npy中，直接加载即可
        print('Loading input CSI data of ' + 'Case ' + Didx)
        H = np.load(csi_file_npy)  # 后续可以直接load数据文件
        print("Loading Channel CSI from npy succeed",H[:1])
    else:
        print("Channel CSI not exist. Reading data from origin txt.")
        H = dataset.read_channel_csi(csi_file_txt,tol_samp_num, sc_num, ant_num, port_num)
        H = H.astype(np.complex64)  # 默认读取为Complex128精度，转换为Complex64降低存储开销
        print("Loading Channel CSI from txt succeed",H[:1])
        # 保存 CSI 为 npy
        np.save(csi_file_npy, H)
        print('Channel CSI saved: '+csi_file_npy)
    del H  # 释放中间结果内存


Processing Round 1 Case 1
Loading configuration data file
Loading input position file
Loading input CSI data of Case 1
Loading Channel CSI from npy succeed [[[[ 0.561971-1.049267j -0.622792-1.145323j -1.293892-0.051079j ...
     1.011706+0.210886j  0.871181-0.794395j -0.241396-1.18387j ]
   [ 0.735801-0.694476j -0.163119-1.083503j -1.076156-0.409428j ...
     0.881897+0.433678j  0.96683 -0.418212j  0.140825-1.122207j]
   [ 0.695794-0.406159j  0.082528-0.862974j -0.773521-0.58058j  ...
     0.654048+0.583802j  0.841062-0.123928j  0.41193 -0.890054j]
   ...
   [-0.290386+0.887194j  0.644633+0.723112j  1.044301-0.125969j ...
    -0.62115 -0.114833j -0.465935+0.488717j  0.149184+0.710186j]
   [-0.517513+0.782341j  0.433433+0.911473j  1.024782+0.175269j ...
    -0.538531-0.143888j -0.469424+0.401872j  0.041793+0.657006j]
   [-0.762837+0.594997j  0.098281+1.040555j  0.937581+0.493241j ...
    -0.587722-0.216617j -0.587406+0.356336j -0.071844+0.69405j ]]

  [[ 0.740383-0.259481j  0.135178-0.7